# Least Squares, Uniqueness, and Intro to Regularization

Graduate Quantitative Economics and Datascience

Jesse Perla (University of British Columbia)

# Overview

## Motivation

-   In this section we will use some of the previous tools and discuss
    concepts on the curvature of optimization problems
-   Doing so, we will consider uniqueness in optimization problems in
    datascience, economics, and ML
-   Our key optimization problems to consider will be the quadratic
    problems than come out of least squares regressions.
    -   However, our discussion of curvature will also provide insights
        into more general optimization problems.

## Packages

This section uses the following packages:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from numpy.linalg import cond, matrix_rank, norm
from scipy.linalg import inv, solve, det, eig, lu, eigvals
from scipy.linalg import solve_triangular, eigvalsh, cholesky

## Reminder: Positive Definite

In [3]:
A = np.array([[3, 1], [1, 2]])
# A_eigs = np.real(eigvals(A)) # symmetric matrices have real eigenvalues
A_eigs = eigvalsh(A) # specialized for symmetric/hermitian matrices
print(A_eigs)
is_positive_definite = np.all(A_eigs > 0)
is_positive_semi_definite = np.all(A_eigs >= 0) # or eigvals(A) >= -eps
print(f"pos-def? {is_positive_definite}")
print(f"pos-semi-def? {is_positive_semi_definite}")

[1.38196601 3.61803399]
pos-def? True
pos-semi-def? True

## Reminder: Positive Definite

In [4]:
A = np.array([[3, -0.5], [-0.1, 2]])
# A_eigs = np.real(eigvals(A)) # symmetric matrices have real eigenvalues
A_eigs = eigvalsh(A) # specialized for symmetric/hermitian matrices
print(A_eigs)
is_positive_definite = np.all(A_eigs > 0)
is_positive_semi_definite = np.all(A_eigs >= 0) # or eigvals(A) >= -eps
print(f"pos-def? {is_positive_definite}")
print(f"pos-semi-def? {is_positive_semi_definite}")

[1.99009805 3.00990195]
pos-def? True
pos-semi-def? True

## Reminder: Positive Semi-Definite Matrices

The simplest positive-semi-definite (but not posdef) matrix is

In [5]:
A_eigs = eigvalsh(np.array([[1, 0], [0, 0]]))
print(A_eigs)
is_positive_definite = np.all(A_eigs > 0)
is_positive_semi_definite = np.all(A_eigs >= 0) # or eigvals(A) >= -eps
print(f"pos-def? {is_positive_definite}")
print(f"pos-semi-def? {is_positive_semi_definite}")

[0. 1.]
pos-def? False
pos-semi-def? True

## Negative Definite Matrices

-   Simply swap the inequality. Think of a convex vs. concave function

In [6]:
A = -1 * np.array([[3, -0.5], [-0.1, 2]])
A_eigs = eigvalsh(A)
print(A_eigs)
is_negative_definite = np.all(A_eigs < 0)
is_negative_semi_definite = np.all(A_eigs <= 0) # or eigvals(A) >= -eps
print(f"neg-def? {is_negative_definite}, neg-semi-def? {is_negative_semi_definite}")

[-3.00990195 -1.99009805]
neg-def? True, neg-semi-def? True

## Negative Definite Matrix

-   Semi-definite, but not definite requires the matrix to not be full
    rank (i.e., an eigenvalue of 0)

In [7]:
A = np.array([[-1, -1], [-1, -1]])
A_eigs = eigvalsh(A)
print(A_eigs)
is_negative_definite = np.all(A_eigs < 0)
is_negative_semi_definite = np.all(A_eigs <= 0) # or eigvals(A) >= -eps
print(f"neg-def? {is_negative_definite}, neg-semi-def? {is_negative_semi_definite}")

[-2.  0.]
neg-def? False, neg-semi-def? True

# Least Squares and the Normal Equations

## Least Squares

Given a matrix $X \in \mathbb{R}^{N \times M}$ and a vector
$y \in \mathbb{R}^N$, we want to find $\beta \in \mathbb{R}^M$ such that
$$
\begin{aligned}
\min_{\beta} &||y - X \beta||^2, \text{ that is,}\\
\min_{\beta} &\sum_{n=1}^N \frac{1}{N}(y_n - X_n \cdot \beta)^2
\end{aligned}
$$

Where $X_n$ is n’th row. Take FOCS and rearrange to get

$$
(X^T X) \beta =  X^T y
$$

## Solving the Normal Equations

-   The $X$ is often referred to as the “design matrix”. $X^T X$ as the
    Gram matrix

-   Can form $A = X^T X$ and $b = X^T y$ and solve $A \beta = b$.

    -   Or invert $X^T X$ to get

    $$
    \beta = (X^T X)^{-1} X^T y
    $$

    -   Note that $X^T X$ is symmetric and, if $X$ is full-rank,
        positive definite

## Solving Regression Models in Practice

-   In practice, use the `lstsq` function in scipy
    -   It uses better algorithms using eigenvectors. More stable (see
        next lecture on conditioning)
    -   One algorithm uses another factoring, the QR decomposition
    -   There, $X = Q R$ for $Q$ orthogonal and $R$ upper triangular.
        See [QR
        Decomposition](https://python.quantecon.org/qr_decomp.html) for
        more
-   Better yet, for applied work use higher-level libraries like
    `statsmodels` (integrates well with `pandas` and `seaborn`)
    -   See [statsmodels
        docs](https://www.statsmodels.org/dev/example_formulas.html) for
        R-style notation
    -   See [QuantEcon OLS Notes](https://python.quantecon.org/ols.html)
        for more.

## Example of LLS using Scipy

In [8]:
N, M = 100, 5
X = np.random.randn(N, M)
beta = np.random.randn(M)
y = X @ beta + 0.05 * np.random.randn(N)
beta_hat, residuals, rank, s = scipy.linalg.lstsq(X, y)
print(f"beta =\n {beta}\nbeta_hat =\n{beta_hat}")

beta =
 [ 0.96527585 -0.64393442  0.70843195  0.02742802 -1.44379448]
beta_hat =
[ 0.96139586 -0.63943654  0.71766934  0.03237551 -1.4455064 ]

## Solving using the Normal Equations

Or we can solve it directly. Provide matrix structure (so it can use a
Cholesky)

In [9]:
beta_hat = solve(X.T @ X, X.T @ y, assume_a="pos")
print(f"beta =\n {beta}\nbeta_hat =\n{beta_hat}")

beta =
 [ 0.96527585 -0.64393442  0.70843195  0.02742802 -1.44379448]
beta_hat =
[ 0.96139586 -0.63943654  0.71766934  0.03237551 -1.4455064 ]

## Collinearity in “Tall” Matrices

-   Tall $\mathbb{R}^{N\times M}$ “design matrices” have $N > M$ and are
    “overdetermined”
-   The rank of a matrix is full rank if all columns are linearly
    independent
-   You can only identify $M$ parameters with $M$ linearly independent
    columns

In [10]:
X = np.array([[1, 2], [2, 5], [3, 7]]) # 3 observations, 2 variables
X_col = np.array([[1, 2], [2, 4], [3, 6]]) # all proportional
print(f"rank(X) = {matrix_rank(X)}, rank(X_col) = {matrix_rank(X_col)}")

rank(X) = 2, rank(X_col) = 1

## Collinearity and Estimation

-   If $X$ is not full rank, then $X^T X$ is not invertible. For
    example:

In [11]:
print(f"cond(X'*X)={cond(X.T@X)}, cond(X_col'*X_col)={cond(X_col.T@X_col)}")

cond(X'*X)=2819.332978639814, cond(X_col'*X_col)=1.1014450683078442e+16

-   Note that when you start doing operations on matrices, numerical
    error creeps in, so you will not get an exact number
-   The rule-of-thumb with condition numbers is that if it is
    $1\times 10^k$ then you lose about $k$ digits of precision. So this
    effectively means it is singular
-   Given the singular matrix, this means a continuum of $\beta$ will
    solve the problem

## `lstsq` Solves it? Careful on Interpretation!

-   Since $X_{col}^T X_{col}$ is singular, we cannot use
    `solve(X.T@X, y)`
-   But what about `lstsq` methods?
-   As you will see, this gives an answer. Interpretation is hard
-   The key is that in the case of non-full rank, you cannot identify
    individual parameters
    -   Related to “Identification” in econometrics
    -   Having low residuals is not enough

In [12]:
y = np.array([5.0, 10.1, 14.9])
beta_hat, residuals, rank, s = scipy.linalg.lstsq(X_col, y)
print(f"beta_hat_col = {beta_hat}")
print(f"rank={rank}, cols={X.shape[1]}, norm(X*beta_hat_col-y)={norm(residuals)}")

beta_hat_col = [0.99857143 1.99714286]
rank=1, cols=2, norm(X*beta_hat_col-y)=0.0

## Fat Design Matrices

-   Fat $\mathbb{R}^{N\times M}$ “design matrices” have $N < M$ and are
    “underdetermined”
-   Less common in econometrics, but useful to understand the structure
-   A continuum $\beta\in\mathbb{R}^{M - \text{rank}(X)}$ solve this
    problem

In [13]:
X = np.array([[1, 2, 3], [0, 5, 7]]) # 2 rows, 3 variables
y = np.array([5, 10])
beta_hat, residuals, rank, s = scipy.linalg.lstsq(X, y)
print(f"beta_hat = {beta_hat}, rank={rank}, ? residuals = {residuals}")

beta_hat = [0.8 0.6 1. ], rank=2, ? residuals = []

## Which Solution?

-   Residuals are zero here because there are enough parameters to fit
    perfectly (i.e., it is underdetermined)
-   Given the multiple solutions, the `lstsq` is giving

$$
\min_{\beta} ||\beta||_2^2 \text{ s.t. } X \beta = y
$$

-   i.e., the “smallest” coefficients which interpolate the data exactly
-   Which trivially fulfills the OLS objective:
    $\min_{\beta} ||y - X \beta||_2^2$

## Careful Interpreting Underdetermined Solutions

-   Useful and common in ML, but be **very** careful when interpreting
    for economics
    -   Tight connections to Bayesian versions of statistical tests
    -   But until you understand econometrics and “identification” well,
        **stick to full-rank matrices**
    -   **Advanced topics:** search for “Regularization”, “Ridgeless
        Regression” and “Benign Overfitting in Linear Regression.”